# POC of model finetuning using AISG FAQ data
Note: this is not a rigorous proof of the finetuning abilities, it merely shows that we are able to overfit to one dataset. The main purpose is to build and test the finetuning code using tensorflow.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from src.utils import read_txt, split_txt, aiap_qna_quickscore
import tensorflow as tf
import tensorflow_hub as hub

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from src.model import QnaEncoderModel
model=QnaEncoderModel()

Instructions for updating:
Colocations handled automatically by placer.


W0920 09:09:06.039438 140447961872128 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0920 09:09:21.475511 140447961872128 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0920 09:09:28.055777 140447961872128 saver.py:1483] Saver not created because there are no variables in the graph to restore


AttributeError: module 'tensorflow._api.v1.losses' has no attribute 'metric_learning'

In [ ]:
aiap_qa, aiap_context = split_txt(read_txt('./data/aiap.txt'), qa=True)

In [ ]:
print(aiap_context)

In [ ]:
aiap_qna_quickscore(aiap_context, model.predict(aiap_qa), aiap_qa, model, 1)

In [ ]:
for _ in range(5):
    for answer, question in zip(aiap_qa, aiap_context):
#         print(answer, question)
        model.finetune([question], [answer], [answer])

In [ ]:
aiap_qna_quickscore(aiap_context, model.predict(aiap_qa), aiap_qa, model, 1)

5 epoch 0.7

# Fine tuning on toy dataset works! Now to try for insurance dataset
This is more rigorous, cause I will be testing on unseen data.
## First load the test framework for qna


In [ ]:
from src.utils import question_cleaner, display_qn_and_ans
from pathlib import Path

In [ ]:
datapath=Path('./data')
df_query = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.question.anslabel.raw.100.pool.solr.train.encoded', delimiter='\t', header=None)
df_doc = pd.read_csv(datapath/'insuranceQA/V2/InsuranceQA.label2answer.raw.encoded', delimiter='\t', header=None)
df_ind2word = pd.read_csv(datapath/'insuranceQA/V2/vocabulary', sep='\t', header=None, quotechar='', quoting=3, keep_default_na=False)
dict_ind2word = pd.Series(df_ind2word[1].values,index=df_ind2word[0].values).to_dict()

In [ ]:
df_query=question_cleaner(df_query)
df_doc=df_doc.set_index(0)

In [ ]:
def wordifier(tokes):
    return ' '.join([dict_ind2word[ind] for ind in tokes.strip().split(' ')])
df_doc['text']=df_doc.apply(lambda x: wordifier(x[1]), axis=1)
df_query['text']=df_query.apply(lambda x: wordifier(x[1]), axis=1)
display(df_query.head(1))
display(df_doc.head(1))

In [ ]:
display_qn_and_ans(df_query, df_doc, index=0)